A partir de las 3 tablas ya procesdas : 
* homeantennas
* sumlinks
* groundtruth

Procedemos a utilizar los clasificadores para ver prediccion

In [1]:
## seteamos los parametros del notebook
%autosave 180
import pandas as pd; 
import numpy as np; 
import os;
import random;
import time
%matplotlib inline
import numpy as np; import os;import random;
import graphlab as gl
#esto es para dibujar directo a la notebook
gl.canvas.set_target('ipynb')

#seteamos el lugar de trabajo
#seteamos el lugar de trabajo
HOMEDIR=os.path.expanduser('~')

PROJECTDIR = os.getcwd().split(os.sep)
PROJECTDIR =  os.sep.join(PROJECTDIR[:PROJECTDIR.index('mexico-scripts-ver2') + 1])

DATADIR = os.path.join(PROJECTDIR,'datasets')

DATADIR2 = os.path.join(PROJECTDIR,'data')

# os.chdir(DATADIR)


Autosaving every 180 seconds


In [31]:
!ls $DATADIR

gtruth_0114_0715  gtruth_0215_0715  homeant  sl  sl.csv.gz


In [2]:
def get_input_sframe(ds):
    if ds == 'home_antenna': url = DATADIR +"/homeant"
    elif ds == 'sum_links': url  = DATADIR +"/sl"
    elif ds == 'gtruth_02': url  = DATADIR +"/gtruth_0215_0715" # can be near ground truth (previous to july 2015)
    elif ds == 'gtruth_01': url  =  DATADIR +"/gtruth_0114_0715" # or can be old GT
    else: print('type chosen is %s, type should be home_antenna, sum_links, gtruth_02 or gtruth_01' % ds)
    return url 

In [3]:
url = get_input_sframe('gtruth_01')
!du -sh $url

92M	/home/juan/mobility-study/mexico-scripts-ver2/datasets/gtruth_0114_0715


In [4]:
#from unbalanced_dataset import UnderSampler, NearMiss, CondensedNearestNeighbour, OneSidedSelection,\
#NeighbourhoodCleaningRule, TomekLinks, ClusterCentroids, OverSampler, SMOTE,\
#SMOTETomek, SMOTEENN, EasyEnsemble, BalanceCascade

In [ ]:
## load a reduced sized

In [1]:
# set what is the resulting sample we want of the global dataset
global_split = 0.25
global_split = None
# since we have 2 datseets that will be later joined and which are going to be previously sampled, in the end this 
# independent sampling will result in that, after the join, the dataset will have a size of fraction given by 
# the `global_split` variable

seed = 2015
if global_split:
    sample = np.sqrt(global_split)
    print(sample)

In [9]:
%%time

sf_table = gl.load_sframe(get_input_sframe('home_antenna'))
sl_table = gl.load_sframe(get_input_sframe('sum_links'))
gt_table = gl.load_sframe(get_input_sframe('gtruth_01'))


if global_split:
    sf_table, _ = sf_table.random_split(sample,seed=seed)
    sl_table, _ = sl_table.random_split(sample,seed=seed)
    gt_table, _ = gt_table.random_split(sample,seed=seed)

rename_gt = (dict([(col,col+"_gt") for col in gt_table.column_names() if col != 'USER']))

#agrego la etiqueta "_gt" a las columnas del ground_truth
gt_table.rename(rename_gt) 

CPU times: user 20 ms, sys: 4 ms, total: 24 ms
Wall time: 207 ms


# checking table consistency
i.e. that users intersect across datasets

In [10]:
%%time
gt_users = set(gt_table['USER'].to_numpy())
sl_users = set(sl_table['USER'].to_numpy())
sf_users = set(sf_table['USER'].to_numpy())

CPU times: user 42.8 s, sys: 3.93 s, total: 46.7 s
Wall time: 1min 16s


In [11]:
%%time
print(sf_table.shape, sl_table.shape, gt_table.shape)
print(len(gt_users), len(sl_users), len(sf_users))

((2035391, 43), (1787143, 131), (2341136, 3))
(2341136, 1787143, 2035391)
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 6.54 ms


In [12]:
%%time
len(gt_users & sl_users), len(gt_users & sf_users), len(sf_users & sl_users)

CPU times: user 4.32 s, sys: 176 ms, total: 4.49 s
Wall time: 6.48 s


(1483010, 1628968, 1787143)

## simple format description

Aca miramos las top 10 antenas que utilizo un usuario, ordeandas de 0 a 10 segun el uso, en numero de llamados, que el user le daba a c. antenna. _0_ Seria la antenna mas utilizada y _10_ la menos utilizada. El count es la cantidad de veces que utilizo esa antenna. 

Cuando aparece _WEEKNIGHT_ en el nombre de la columna, es porque cumple la condicion de que los llamados fueron hechos de noche fuera del horario [8,20] y dentro de la semana laboral.

Siguiendo las definiciones del trabajo de Caro, un user es _EPIDEMIC_ siii su ANTENNA_WEEKNIGHT_0 (esta es la home_antenna) pertence a la zona epidemica.

El mobility_diameter es el radio de las antennas (0 si uso una sola, etc.) utilizadas por este user. Nuevamente el modificador _WEEKNIGHT_ solo aplica para antennas utilizadas en esos horarios.

In [13]:
# column names
sf_table.column_names()

['USER',
 'COUNT_0',
 'COUNT_1',
 'COUNT_2',
 'COUNT_3',
 'COUNT_4',
 'COUNT_5',
 'COUNT_6',
 'COUNT_7',
 'COUNT_8',
 'COUNT_9',
 'ANTENNA_ID_0',
 'ANTENNA_ID_1',
 'ANTENNA_ID_2',
 'ANTENNA_ID_3',
 'ANTENNA_ID_4',
 'ANTENNA_ID_5',
 'ANTENNA_ID_6',
 'ANTENNA_ID_7',
 'ANTENNA_ID_8',
 'ANTENNA_ID_9',
 'COUNT_WEEKNIGHT_0',
 'COUNT_WEEKNIGHT_1',
 'COUNT_WEEKNIGHT_2',
 'COUNT_WEEKNIGHT_3',
 'COUNT_WEEKNIGHT_4',
 'COUNT_WEEKNIGHT_5',
 'COUNT_WEEKNIGHT_6',
 'COUNT_WEEKNIGHT_7',
 'COUNT_WEEKNIGHT_8',
 'COUNT_WEEKNIGHT_9',
 'ANTENNA_ID_WEEKNIGHT_0',
 'ANTENNA_ID_WEEKNIGHT_1',
 'ANTENNA_ID_WEEKNIGHT_2',
 'ANTENNA_ID_WEEKNIGHT_3',
 'ANTENNA_ID_WEEKNIGHT_4',
 'ANTENNA_ID_WEEKNIGHT_5',
 'ANTENNA_ID_WEEKNIGHT_6',
 'ANTENNA_ID_WEEKNIGHT_7',
 'ANTENNA_ID_WEEKNIGHT_8',
 'ANTENNA_ID_WEEKNIGHT_9',
 'MOBILITY_DIAMETER',
 'MOBILITY_DIAMETER_WEEKNIGHT']

## sum links description

La tabla sum_links tiene mas atributos y con data mas rica en informacion:
    
Esencialmente para c/ user miramos la cantidad de llamados y el tiempo que duraron esos llamados pero segmentando con distintos modificadores. `Time` representa la duracion del llamado y Calls el conteo de llamados.

Los modficadores/ segmentaciones son:

* mes en el cual estamos parados (12 == diciembre, ..., 08 == agosto)
* OUT/IN, separa por la direccion de los llamados si salientes u entrantes.
* _VULN_ : separa los llamados que fueron realizados hacia/desde un target_user (en una llamada hay 2 usuarios, el origin o el target) viviendo en una zona epidemica. Donde la home antena de un target_user determina su vulnerabilidad segun si es zona epidemica o no.
* Weekend, WeekDay y WeekNight son lo que suenan. Weekend el finde, Weeknight la semana pero fuera de horario laboral y Weekday en horario laboral y de lunes a viernes.

Hay solo una columna que no entra enteramente en este esquema que es VULNERABLE. Esta columna hace un conteo p/c/ usuario d cuantos target_users viven en una zona epidemica. Tambien se segmenta esta columna con los modficiadores anteriores (el mes y el out/in).

In [14]:
# column names
sl_table.column_names()

['USER',
 'TimeWeekEnd_OUT_12',
 'CallsWeekDay_OUT_12',
 'CallsWeekNight_OUT_12',
 'TimeWeekDay_OUT_12',
 'CallsWeekEnd_OUT_12',
 'TimeWeekNight_OUT_12',
 'TimeWeekEnd_OUT_VUL_12',
 'CallsWeekDay_OUT_VUL_12',
 'CallsWeekNight_OUT_VUL_12',
 'TimeWeekDay_OUT_VUL_12',
 'CallsWeekEnd_OUT_VUL_12',
 'TimeWeekNight_OUT_VUL_12',
 'VULNERABLE_OUT_12',
 'TimeWeekEnd_IN_12',
 'CallsWeekDay_IN_12',
 'CallsWeekNight_IN_12',
 'TimeWeekDay_IN_12',
 'CallsWeekEnd_IN_12',
 'TimeWeekNight_IN_12',
 'TimeWeekEnd_IN_VUL_12',
 'CallsWeekDay_IN_VUL_12',
 'CallsWeekNight_IN_VUL_12',
 'TimeWeekDay_IN_VUL_12',
 'CallsWeekEnd_IN_VUL_12',
 'TimeWeekNight_IN_VUL_12',
 'VULNERABLE_IN_12',
 'TimeWeekEnd_OUT_08',
 'CallsWeekDay_OUT_08',
 'CallsWeekNight_OUT_08',
 'TimeWeekDay_OUT_08',
 'CallsWeekEnd_OUT_08',
 'TimeWeekNight_OUT_08',
 'TimeWeekEnd_OUT_VUL_08',
 'CallsWeekDay_OUT_VUL_08',
 'CallsWeekNight_OUT_VUL_08',
 'TimeWeekDay_OUT_VUL_08',
 'CallsWeekEnd_OUT_VUL_08',
 'TimeWeekNight_OUT_VUL_08',
 'VULNERABLE_OUT_0

# Para la
tabla gt (ground_truth) es mas simple la explicacion. Solo se busco el antenna_ID_0 (nuevamente la antenna mas utilizada) por un user

In [15]:
gt_table.head()

USER,ANTENNA_ID_0_gt,ANTENNA_ID_WEEKNIGHT_0_gt
41D91B8CB8747902A80D6108D548A8AF ...,2898,2898
24A624FC979DF41830A31C7CED65E2D2 ...,4285,4285
C3FF4C9281746CAC6C09802C254BDE71 ...,4336,4336
212D3A9745F1347063267844311B31FF ...,1200,884
544695294E816F26691C4D01516ABF9B ...,195,195
FEC4BD059A42FB5283E07539C1FD2CB1 ...,262,262
BDEF57DB3EF1035FF3F210B47FC2A5B0 ...,1557,4021
3AE1DCA333E55CA87E4F5331035DB600 ...,608,608
2A72E07E48667A6289D93874A623C152 ...,2302,2302
C5287E4004BE8F3A12AD489FE8204056 ...,2644,828


# Sanity Checks 

not all users are moving away from the epidemic zone..

## Sanity check data Consistency
Let $X$ be the matrix describing the features of our models and $Y$ our ground truth.

* Do we have the same antenna_id between X and Y?
* How much are users 'swapping' past/present home_antennas? swapping past/present epidemic status?

### matriz de confusion (contingencia)
http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html


By definition a confusion matrix $C$ is such that $C_{i, j}$

is equal to the number of observations known to be in group $i $ but
predicted to be in group $j$.
    
Como leer:
* da el # de elementos en una categoria 
* La Prediccion va como columnas (eje 1)
* La condicion real corre como filas (eje 0)

|  	| No Pred 	| Yes Pred 	|
|:-------:	|----------------	|----------------	|
| Actual No 	| True Negative 	| False Negative 	|
| Actual Yes 	| False Positive 	| True Positve 	|


# We need antennas metadata

In [16]:
#agregamos al gt su info de epidemicidad
antennas_file = DATADIR2+'/celdas_limpio.csv'
antennas = gl.SFrame.read_csv(antennas_file, 
                              delimiter= "|", 
                              usecols=['LATITUDE','LONGITUDE','CEL_ID','STATE','EPIDEMIC'],
                              column_type_hints=[float, float, str,str, bool])

Finished parsing file /home/juan/mobility-study/mexico-scripts-ver2/data/celdas_limpio.csv

Parsing completed. Parsed 4415 lines in 0.11035 secs.

In [17]:
# agrego tambien data de epidemicidad al simpleformat table
sf_table = sf_table.join(antennas['CEL_ID','EPIDEMIC','STATE'], 
                                     on = {'ANTENNA_ID_WEEKNIGHT_0':'CEL_ID'},
                                     how = 'left')

In [18]:
%%time
# enriquecemos la data de gt con info de epidemicidad
gt_table = gt_table.join(antennas['CEL_ID','EPIDEMIC','STATE'], 
                         on = {'ANTENNA_ID_WEEKNIGHT_0_gt':'CEL_ID'},
                         how = 'left')

gt_table.rename({'EPIDEMIC':'EPIDEMIC_gt'})

CPU times: user 17.9 s, sys: 284 ms, total: 18.2 s
Wall time: 4.51 s


In [20]:
# count antenna distribution per state
a = antennas.to_dataframe()['STATE'].value_counts().head(10)
(a/a.sum()).head(15)

Distrito_Federal    0.282549
Mexico              0.212679
Jalisco             0.107703
Nuevo_Leon          0.096796
Guanajuato          0.060668
Puebla              0.058623
Veracruz            0.054192
Baja_California     0.043286
Yucatan             0.041922
Sinaloa             0.041581
Name: STATE, dtype: float64

In [21]:
antennas.head()

LATITUDE,LONGITUDE,CEL_ID,STATE,EPIDEMIC
24.14206333,-110.2940553,1,Baja_California_Sur,0
19.31787222,-99.13700278,2,Distrito_Federal,0
17.87924083,-92.48047778,3,Tabasco,1
22.415,-97.938,4,Tamaulipas,0
19.06744444,-98.221,5,Puebla,1
18.82805556,-99.24444444,6,Morelos,1
17.9997,-92.9282,7,Tabasco,1
25.80656932,-100.3256526,8,Nuevo_Leon,0
19.4906,-99.2712,9,Mexico,0
20.58142655,-103.4326277,10,Jalisco,1


In [22]:
gt_table.column_names()

['USER',
 'ANTENNA_ID_0_gt',
 'ANTENNA_ID_WEEKNIGHT_0_gt',
 'EPIDEMIC_gt',
 'STATE']

In [23]:
# percentage of users per state
a = gt_table['USER','STATE'].groupby('STATE', 
                        {'USERS':gl.aggregate.COUNT('USER')}).to_dataframe()
a['USERS'] = a['USERS']/a['USERS'].sum()
a.sort_values('USERS',ascending=False).head(30)

,STATE,USERS
3,Mexico,0.238536
27,Distrito_Federal,0.200632
29,Jalisco,0.083018
22,Guanajuato,0.059031
12,Puebla,0.043403
5,Veracruz,0.042567
24,Hidalgo,0.033181
11,Yucatan,0.029301
17,Nuevo_Leon,0.029050
18,Morelos,0.027125


In [24]:
gt_table.head()

USER,ANTENNA_ID_0_gt,ANTENNA_ID_WEEKNIGHT_0_gt,EPIDEMIC_gt,STATE
41D91B8CB8747902A80D6108D548A8AF ...,2898,2898,1,Oaxaca
24A624FC979DF41830A31C7CED65E2D2 ...,4285,4285,0,Mexico
C3FF4C9281746CAC6C09802C254BDE71 ...,4336,4336,0,Mexico
212D3A9745F1347063267844311B31FF ...,1200,884,1,Jalisco
544695294E816F26691C4D01516ABF9B ...,195,195,0,Chihuahua
FEC4BD059A42FB5283E07539C1FD2CB1 ...,262,262,0,Guanajuato
BDEF57DB3EF1035FF3F210B47FC2A5B0 ...,1557,4021,1,Tabasco
3AE1DCA333E55CA87E4F5331035DB600 ...,608,608,0,Mexico
2A72E07E48667A6289D93874A623C152 ...,2302,2302,0,Distrito_Federal
C5287E4004BE8F3A12AD489FE8204056 ...,2644,828,0,Queretaro


# fast check how many users have same/different antennas in the past

In [25]:
past_user_data = sf_table['USER','ANTENNA_ID_WEEKNIGHT_0', 'ANTENNA_ID_0','EPIDEMIC'].join(\
                            gt_table['USER','ANTENNA_ID_WEEKNIGHT_0_gt', 'ANTENNA_ID_0_gt','EPIDEMIC_gt'], 
                            on= 'USER')

In [26]:
#inherentemente vamos a perder algunos users sin past data del home_antenna
print(past_user_data.shape)
past_user_data = past_user_data[past_user_data['ANTENNA_ID_WEEKNIGHT_0','ANTENNA_ID_WEEKNIGHT_0_gt'].\
                                apply(lambda x: x['ANTENNA_ID_WEEKNIGHT_0'] != "-1" and \
                                    x['ANTENNA_ID_WEEKNIGHT_0_gt'] != "-1" )]

past_user_data = past_user_data.dropna(columns = ['EPIDEMIC', 'EPIDEMIC_gt'],
                                       how='any')
print(past_user_data.shape)

(1628968, 7)
(1592841, 7)


In [45]:
comparison_dict = {
    'top_antenna':{
            'col1':'ANTENNA_ID_0_gt',
            'col2':'ANTENNA_ID_0',
    },
    'top_week_antenna':{
            'col1':'ANTENNA_ID_WEEKNIGHT_0_gt',
            'col2':'ANTENNA_ID_WEEKNIGHT_0',
    },
    'epidemic':{
            'col1':'EPIDEMIC_gt',
            'col2':'EPIDEMIC',
    },
    
}

# compare = 'top_week_antenna'
compare = 'epidemic'


col1 = comparison_dict[compare]['col1']
col2 = comparison_dict[compare]['col2']
col1,col2

('EPIDEMIC_gt', 'EPIDEMIC')

In [50]:
(past_user_data[col1] == past_user_data[col2]).sum(), (past_user_data[col1] != past_user_data[col2]).sum()

(1551560, 41281)

In [51]:
past_user_data.shape

(1592841, 7)

In [52]:
from sklearn.metrics import confusion_matrix

y_true = past_user_data[col1].to_numpy()
y_pred = past_user_data[col2].to_numpy()
cm = confusion_matrix(y_true = y_true, y_pred = y_pred  )

print('Confusion matrix')
print(cm)


Confusion matrix
[[1140360   18330]
 [  22951  411200]]


# Results: :)

In [53]:
# percentage of people actual non vulnerable that were vulnerable before?
(cm[0,1])*1.0/ cm[0,:].sum()

0.015819589363850556

In [54]:
# percentage of people that migrated?
(cm[0,1] + cm[1,0])*1.0/ cm.sum().sum()

0.02591658552234655

## construct a full sized dataset

In [ ]:
%%time
## integrate target to single frame
data = sf_table.join(gt_table['USER','EPIDEMIC_gt'], 
                     on = 'USER', 
                     how = 'inner')

data = data.join(sl_table, 
                 on = 'USER', 
                 how = 'inner')

#no podemos tener nulls en el target asi que dropeamos
data = data.dropna(columns = ['EPIDEMIC_gt'], 
                   how='any')

print(data.shape)
del sl_table, sf_table, gt_table

In [ ]:
# check certain columns
data.column_names().sample(0.15)

## check correlation and null variance

as a simple method to delete noisy/bad cols

In [ ]:
corr_matrix = abs(data.to_dataframe().corr())
corr_matrix.head()

In [ ]:
view = corr_matrix[corr_matrix['EPIDEMIC_gt']>0.25]['EPIDEMIC_gt'].sort_values(ascending=False)
view